**QUESTION:**

Multivariate calibration of spectral measurements is a technique that is used in chemometrics to develop a model relating spectral measurements (obtained using instruments such as UV, FIR or NIR or MS spectrophotometers) to properties such as concentration or other properties of species (usually liquid or gases). The application we consider is to obtain a model relating UV absorbance spectra to compositions (concentrations) of mixtures. Such a model is useful in online monitoring of chemical and biochemical reactions. Twenty six samples of different concentrations of a mixture of Co, Cr, and Ni ions in dilute nitric acid were prepared in a laboratory and their spectra recorded over the range 300-650 nm using a HP 8452 UV diode array spectrophotometer (data in Inorfull.mat). (Water and ethanol are generally used as solvents since these do not absorb in the UV range. Also the nitrate ions do not absorb in the UV range. So an aqueous solution of nitric acid is used to dissolve the metals in this experiment). Five replicates for each mixture were obtained. The measurements were made at 2 nm intervals giving rise to an absorbance matrix of size 130 x 176. The concentrations of the 26 samples, which is a 26 x 3 matrix are also given in the data file. In order to predict the concentration of the mixture using absorbance measurements, it is necessary to build a calibration model relating concentration of mixtures to its absorbance spectra. According to Beer-Lambert’s law the absorbance spectra of a dilute mixture is a linear (weighted) combination of the pure component spectra with the weights corresponding to the concentrations of the species in the mixture.

If absorbances are measured only a minimum number of wavelengths, then OLS can be used to build a calibration model. For example, if a mixture containing ns non-reacting species, then absorbances at ns wavelengths need to be measured. Typically, the wavelengths are chosen corresponding to the maximum absorbing wavelengths of individual species. However, if we measure absorbances at nw > ns wavelengths, then the absorbance matrix will not be full column rank. In this case, Principal Component Regression can be used to develop a multivariate calibration model. In this method PCA is first applied to the absorbance matrix to obtain the scores corresponding to different mixtures. In the second step, a regression model is used to relate the concentrations to the scores using OLS (assuming concentrations are the dependent variables). In order to use this model for predicting the concentrations of a mixture whose absorbance spectra is given, we first obtain the scores and then use the OLS regression model to predict the concentrations. Note that the true rank of the absorbance matrix is equal to the number of species in the mixture.

1) The quality of the linear calibration model is evaluated using leave-one-sample-out cross validation (LOOCV) and computing the root mean square error (RMSE) in predicting the left out sample concentrations. Pick the first replicate for each mixture to obtain a data matrix of size 26 x 176 and use it for the following different multivariate calibration modelling methods. For each method report the LOOCV RMSE results in the form of a table for number of PCs chosen between 1 and 10. Based on the LOOCV RMSE values indicate whether you are able to estimate the number of species correctly?

2) Instead of LOOCV you can also use k-fold cross validation for selecting number of PCs. Compute RMSE using 5-fold cross validation for various values of PCs from 1 to 10 and check whether correct number of PCs can be determined.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spectra-data/Inorfull.mat


Importing necessary libraries.

In [10]:
import scipy.io
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

loading the dataset.

In [3]:
data=scipy.io.loadmat("/kaggle/input/spectra-data/Inorfull.mat")

Unpacking the dataset.

In [4]:
data.items()

dict_items([('__header__', b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Wed May 29 14:21:55 2002'), ('__version__', '1.0'), ('__globals__', []), ('CONC', array([[0.00305775, 0.01571527, 0.0068796 ],
       [0.00305775, 0.01571527, 0.0068796 ],
       [0.00305775, 0.01571527, 0.0068796 ],
       [0.00305775, 0.01571527, 0.0068796 ],
       [0.00305775, 0.01571527, 0.0068796 ],
       [0.00305775, 0.01571527, 0.02063881],
       [0.00305775, 0.01571527, 0.02063881],
       [0.00305775, 0.01571527, 0.02063881],
       [0.00305775, 0.01571527, 0.02063881],
       [0.00305775, 0.01571527, 0.02063881],
       [0.00305775, 0.01571527, 0.03439802],
       [0.00305775, 0.01571527, 0.03439802],
       [0.00305775, 0.01571527, 0.03439802],
       [0.00305775, 0.01571527, 0.03439802],
       [0.00305775, 0.01571527, 0.03439802],
       [0.00305775, 0.0471458 , 0.0068796 ],
       [0.00305775, 0.0471458 , 0.0068796 ],
       [0.00305775, 0.0471458 , 0.0068796 ],
       [0.00305775, 0.0471458

In [5]:
# printing the given data columns.
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'CONC', 'DATA', 'PureCo', 'PureCr', 'PureNi', 'WAV', 'stdDATA', 'PureCoCONC', 'PureCrCONC', 'PureNiCONC'])


In [6]:
# printing the shape of every data given.
print("CONC",data["CONC"].shape)
print("DATA",data['DATA'].shape)
print("PureCo",data["PureCo"].shape)
print("PureCr",data['PureCr'].shape)
print("PureNi",data['PureNi'].shape)
print("WAV",data['WAV'].shape)
print("stdDATA",data['stdDATA'].shape)
print("PureCoCONC",data["PureCoCONC"].shape)
print("PureCrCONC",data['PureCrCONC'].shape)
print("PureNiCONC",data['PureNiCONC'].shape)

CONC (130, 3)
DATA (130, 176)
PureCo (1, 176)
PureCr (1, 176)
PureNi (1, 176)
WAV (1, 176)
stdDATA (130, 176)
PureCoCONC (1, 1)
PureCrCONC (1, 1)
PureNiCONC (1, 1)


Extracting the absorbance matrix and the concentration matrix from the loaded data.

In [7]:
X=data['DATA']
Y=data['CONC']
std_X=data['stdDATA']

There is no need of standardizing the data because it is already given.

Here, we first load the data separate the features X (spectral measurements) and the target variable y(concentrations). Then, we loop over the number of principal components we want to use (1 to 10), and within each iteration, we perform LOOCV by leaving out one sample at a time and using the remaining samples to train a PCA model and a linear regression model. We then use the trained models to predict the left-out sample's target variable and compute the RMSE. Finally, we take the mean RMSE across all left-out samples and print it along with the number of principal components used.

Based on the LOOCV RMSE values, we can estimate the number of species correctly by looking at the point at which the RMSE values start to plateau or decrease only marginally with increasing number of principal components.

Picking up the first replicate for each mixture to obtain a data matrix of size 26 x 176 and use it for the following different multivariate calibration modelling methods. 

In [8]:
X = data['stdDATA'][:26,:] 
y = data['CONC'][:26,:] 
print(X.shape,y.shape)

(26, 176) (26, 3)


In [9]:
# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# Set up LOOCV
n_samples = X_pca.shape[0]
loo = LeaveOneOut()

# Perform OLS regression with PCA and LOOCV
n_components = 10
rmse_train = np.zeros(n_components)
for i in range(1, n_components + 1):
    X_train_pca = X_pca[:, :i]
    rmse_train_temp = np.zeros(n_samples)
    for train_index, test_index in loo.split(X_train_pca):
        X_train_pca_loo, X_test_pca_loo = X_train_pca[train_index], X_train_pca[test_index]
        y_train_loo, y_test_loo = y[train_index], y[test_index]
        model = LinearRegression()
        model.fit(X_train_pca_loo, y_train_loo)
        y_pred_loo = model.predict(X_test_pca_loo)
        rmse_train_temp[test_index] = mean_squared_error(y_test_loo, y_pred_loo, squared=False)
    rmse_train[i-1] = np.mean(rmse_train_temp)

print("OLS Regression with PCA and LOOCV Results:")
print("Number of Components\tRMSE (LOOCV)")
for i in range(1, n_components + 1):
    print(f"{i}\t\t\t{rmse_train[i-1]:.4f}")


OLS Regression with PCA and LOOCV Results:
Number of Components	RMSE (LOOCV)
1			0.0075
2			0.0057
3			0.0038
4			0.0017
5			0.0006
6			0.0006
7			0.0006
8			0.0006
9			0.0006
10			0.0006


**CONCLUSION 1:**

From the above output,we can clearly see that the RMSE value is constant after 5 components.
So the optimal number of components required are 5. Thus, number of species required are also 5.

Instead of LOOCV you can also use k-fold cross validation for selecting number of PCs. Compute RMSE using 5-fold cross validation for various values of PCs from 1 to 10 and check whether correct number of PCs can be determined.

In [12]:
# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(X)

n_samples = X_pca.shape[0]

# using 5-fold cross validation
cross_val = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform OLS regression with PCA and LOOCV
n_components = 10
rmse_train = np.zeros(n_components)
for i in range(1, n_components + 1):
    X_train_pca = X_pca[:, :i]
    rmse_train_temp = np.zeros(n_samples)
    for j,(train_index, test_index) in enumerate(cross_val.split(X_train_pca)):
        X_train_pca_cv, X_test_pca_cv = X_train_pca[train_index], X_train_pca[test_index]
        y_train_cv, y_test_cv = y[train_index], y[test_index]
        model = LinearRegression()
        model.fit(X_train_pca_cv, y_train_cv)
        y_pred_cv = model.predict(X_test_pca_cv)
        rmse_train_temp[test_index] = mean_squared_error(y_test_cv, y_pred_cv, squared=False)
    rmse_train[i-1] = np.mean(rmse_train_temp)

print("OLS Regression with PCA and 5-FoldCV Results:")
print("Number of Components\tRMSE (5-FoldCV)")
for i in range(1, n_components + 1):
    print(f"{i}\t\t\t{rmse_train[i-1]:.4f}")

OLS Regression with PCA and 5-FoldCV Results:
Number of Components	RMSE (5-FoldCV)
1			0.0079
2			0.0062
3			0.0045
4			0.0028
5			0.0013
6			0.0013
7			0.0013
8			0.0013
9			0.0013
10			0.0013


**CONCLUSION 2:**

From the above output,we can clearly see that the optimal number of components required are 5 even using the 5-fold cross validation method. Thus, number of species required are 5.